# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo SINASC_RO_2019.csv` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo `estrutura_sinasc_para_CD.pdf` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
`
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis `['ESTCIVMAE', 'CONSULTAS']` possuem o código `9`, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código `9`.<br>
6. Substitua os valores faltantes da quantitativa (`QTDFILVIVO`) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção `inplace = True`.

In [255]:
import pandas as pd
import requests

# 1) seu código aqui
sinasc = pd.read_csv('SINASC_RO_2019.csv')
print(sinasc.shape)
sinasc.drop_duplicates().shape
# Não há duplicados

(27028, 69)


(27028, 69)

In [256]:
sinasc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27028 entries, 0 to 27027
Data columns (total 69 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ORIGEM        27028 non-null  int64  
 1   CODESTAB      26913 non-null  float64
 2   CODMUNNASC    27028 non-null  int64  
 3   LOCNASC       27028 non-null  int64  
 4   IDADEMAE      27028 non-null  int64  
 5   ESTCIVMAE     26711 non-null  float64
 6   ESCMAE        26716 non-null  object 
 7   CODOCUPMAE    24121 non-null  float64
 8   QTDFILVIVO    25455 non-null  float64
 9   QTDFILMORT    24930 non-null  float64
 10  CODMUNRES     27028 non-null  int64  
 11  GESTACAO      25796 non-null  object 
 12  GRAVIDEZ      26949 non-null  object 
 13  PARTO         26979 non-null  object 
 14  CONSULTAS     27028 non-null  int64  
 15  DTNASC        27028 non-null  object 
 16  HORANASC      27007 non-null  float64
 17  SEXO          27024 non-null  object 
 18  APGAR1        26932 non-nu

In [257]:
# 2) seu código aqui
def calcula_missing(dataframe):
    for i in dataframe:
        resultado = dataframe[str(i)].isna().sum()
        print(i, ' - ', resultado, f' missing(s). Equivale há {round((resultado/len(dataframe))*100,2)}% dos dados')

calcula_missing(sinasc)


ORIGEM  -  0  missing(s). Equivale há 0.0% dos dados
CODESTAB  -  115  missing(s). Equivale há 0.43% dos dados
CODMUNNASC  -  0  missing(s). Equivale há 0.0% dos dados
LOCNASC  -  0  missing(s). Equivale há 0.0% dos dados
IDADEMAE  -  0  missing(s). Equivale há 0.0% dos dados
ESTCIVMAE  -  317  missing(s). Equivale há 1.17% dos dados
ESCMAE  -  312  missing(s). Equivale há 1.15% dos dados
CODOCUPMAE  -  2907  missing(s). Equivale há 10.76% dos dados
QTDFILVIVO  -  1573  missing(s). Equivale há 5.82% dos dados
QTDFILMORT  -  2098  missing(s). Equivale há 7.76% dos dados
CODMUNRES  -  0  missing(s). Equivale há 0.0% dos dados
GESTACAO  -  1232  missing(s). Equivale há 4.56% dos dados
GRAVIDEZ  -  79  missing(s). Equivale há 0.29% dos dados
PARTO  -  49  missing(s). Equivale há 0.18% dos dados
CONSULTAS  -  0  missing(s). Equivale há 0.0% dos dados
DTNASC  -  0  missing(s). Equivale há 0.0% dos dados
HORANASC  -  21  missing(s). Equivale há 0.08% dos dados
SEXO  -  4  missing(s). Equivale

In [258]:
# 3) seu código aqui
sinasc_nova = sinasc[['LOCNASC','IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']].copy()
sinasc_nova

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0
...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0


In [259]:
calcula_missing(sinasc_nova)

LOCNASC  -  0  missing(s). Equivale há 0.0% dos dados
IDADEMAE  -  0  missing(s). Equivale há 0.0% dos dados
ESTCIVMAE  -  317  missing(s). Equivale há 1.17% dos dados
ESCMAE  -  312  missing(s). Equivale há 1.15% dos dados
QTDFILVIVO  -  1573  missing(s). Equivale há 5.82% dos dados
GESTACAO  -  1232  missing(s). Equivale há 4.56% dos dados
GRAVIDEZ  -  79  missing(s). Equivale há 0.29% dos dados
CONSULTAS  -  0  missing(s). Equivale há 0.0% dos dados
APGAR5  -  103  missing(s). Equivale há 0.38% dos dados


In [260]:
# 4) seu código aqui
sinasc_nova.dropna(inplace=True,subset=['APGAR5']) # Usamos o subset para restringir a remoção dos valores missing apenas que aparecem na coluna "APGAR5"

In [261]:
sinasc_nova

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0
...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0


In [262]:
calcula_missing(sinasc_nova)

LOCNASC  -  0  missing(s). Equivale há 0.0% dos dados
IDADEMAE  -  0  missing(s). Equivale há 0.0% dos dados
ESTCIVMAE  -  315  missing(s). Equivale há 1.17% dos dados
ESCMAE  -  310  missing(s). Equivale há 1.15% dos dados
QTDFILVIVO  -  1566  missing(s). Equivale há 5.82% dos dados
GESTACAO  -  1216  missing(s). Equivale há 4.52% dos dados
GRAVIDEZ  -  76  missing(s). Equivale há 0.28% dos dados
CONSULTAS  -  0  missing(s). Equivale há 0.0% dos dados
APGAR5  -  0  missing(s). Equivale há 0.0% dos dados


In [263]:
# 5) seu código aqui
sinasc_nova.ESTCIVMAE.fillna(9,inplace=True)
sinasc_nova.CONSULTAS.fillna(9,inplace=True)
sinasc_nova

C:\Users\fabio\AppData\Local\Temp\ipykernel_14596\1508799695.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  sinasc_nova.ESTCIVMAE.fillna(9,inplace=True)
C:\Users\fabio\AppData\Local\Temp\ipykernel_14596\1508799695.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0
...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0


In [264]:
calcula_missing(sinasc_nova)

LOCNASC  -  0  missing(s). Equivale há 0.0% dos dados
IDADEMAE  -  0  missing(s). Equivale há 0.0% dos dados
ESTCIVMAE  -  0  missing(s). Equivale há 0.0% dos dados
ESCMAE  -  310  missing(s). Equivale há 1.15% dos dados
QTDFILVIVO  -  1566  missing(s). Equivale há 5.82% dos dados
GESTACAO  -  1216  missing(s). Equivale há 4.52% dos dados
GRAVIDEZ  -  76  missing(s). Equivale há 0.28% dos dados
CONSULTAS  -  0  missing(s). Equivale há 0.0% dos dados
APGAR5  -  0  missing(s). Equivale há 0.0% dos dados


In [265]:
# 6) Seu código aqui
sinasc_nova.QTDFILVIVO.fillna(0,inplace=True)

C:\Users\fabio\AppData\Local\Temp\ipykernel_14596\61961572.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  sinasc_nova.QTDFILVIVO.fillna(0,inplace=True)


In [266]:
sinasc_nova

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0
...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0


In [267]:
calcula_missing(sinasc_nova)

LOCNASC  -  0  missing(s). Equivale há 0.0% dos dados
IDADEMAE  -  0  missing(s). Equivale há 0.0% dos dados
ESTCIVMAE  -  0  missing(s). Equivale há 0.0% dos dados
ESCMAE  -  310  missing(s). Equivale há 1.15% dos dados
QTDFILVIVO  -  0  missing(s). Equivale há 0.0% dos dados
GESTACAO  -  1216  missing(s). Equivale há 4.52% dos dados
GRAVIDEZ  -  76  missing(s). Equivale há 0.28% dos dados
CONSULTAS  -  0  missing(s). Equivale há 0.0% dos dados
APGAR5  -  0  missing(s). Equivale há 0.0% dos dados


In [268]:
# 7) seu código aqui - Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
sinasc_nova 

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0
...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0


Fiz esse chamado para visualizar as informações existentes nos dados. Além disso, na aba anterior, é possivel verificar que há apenas 3 colunas com presença de dados faltantes, logo me limitarei a realizar tratamento desses dados.

In [269]:
sinasc_nova['GRAVIDEZ'].value_counts() 


GRAVIDEZ
Única              26356
Dupla                488
Tríplice e mais        5
Name: count, dtype: int64

para este caso, é possível verificar uma grande diferença entre os 3 tipos de respostas encontrados e por haver poucos dados missing irei colocar os valores missing como "Única"

In [270]:
sinasc_nova.GRAVIDEZ.fillna("Única",inplace=True)

C:\Users\fabio\AppData\Local\Temp\ipykernel_14596\884840228.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  sinasc_nova.GRAVIDEZ.fillna("Única",inplace=True)


In [271]:
calcula_missing(sinasc_nova)

LOCNASC  -  0  missing(s). Equivale há 0.0% dos dados
IDADEMAE  -  0  missing(s). Equivale há 0.0% dos dados
ESTCIVMAE  -  0  missing(s). Equivale há 0.0% dos dados
ESCMAE  -  310  missing(s). Equivale há 1.15% dos dados
QTDFILVIVO  -  0  missing(s). Equivale há 0.0% dos dados
GESTACAO  -  1216  missing(s). Equivale há 4.52% dos dados
GRAVIDEZ  -  0  missing(s). Equivale há 0.0% dos dados
CONSULTAS  -  0  missing(s). Equivale há 0.0% dos dados
APGAR5  -  0  missing(s). Equivale há 0.0% dos dados


In [272]:
sinasc_nova['GESTACAO'].value_counts()

GESTACAO
37 a 41 semanas        22523
32 a 36 semanas         2234
42 semanas e mais        698
28 a 31 semanas          162
22 a 27 semanas           85
Menos de 22 semanas        7
Name: count, dtype: int64

Neste caso, há um número mais significativo de dados faltante (quase 5% do dataframe) e associado ao fato de possuir uma distribuição mais equilibrada entre os tipos de dados, neste caso irei colocar estes dados como o valor 'não preenchido'

In [273]:
sinasc_nova.GESTACAO.fillna("Não Preenchido",inplace=True)

C:\Users\fabio\AppData\Local\Temp\ipykernel_14596\3525212286.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  sinasc_nova.GESTACAO.fillna("Não Preenchido",inplace=True)


In [274]:
calcula_missing(sinasc_nova)

LOCNASC  -  0  missing(s). Equivale há 0.0% dos dados
IDADEMAE  -  0  missing(s). Equivale há 0.0% dos dados
ESTCIVMAE  -  0  missing(s). Equivale há 0.0% dos dados
ESCMAE  -  310  missing(s). Equivale há 1.15% dos dados
QTDFILVIVO  -  0  missing(s). Equivale há 0.0% dos dados
GESTACAO  -  0  missing(s). Equivale há 0.0% dos dados
GRAVIDEZ  -  0  missing(s). Equivale há 0.0% dos dados
CONSULTAS  -  0  missing(s). Equivale há 0.0% dos dados
APGAR5  -  0  missing(s). Equivale há 0.0% dos dados


In [275]:
sinasc_nova['ESCMAE'].value_counts()

ESCMAE
8 a 11 anos        15581
12 anos ou mais     5757
4 a 7 anos          4727
1 a 3 anos           510
Nenhuma               40
Name: count, dtype: int64

Usarei a mesma análise dos dados de gestação, apesar da quantidade ser menor de missings, ainda assim a distribuição existente, pode interferir significativamente caso esses missings recebam valores existentes para o tratamento dos dados.

In [276]:
sinasc_nova.ESCMAE.fillna("Não Preenchido",inplace=True)

C:\Users\fabio\AppData\Local\Temp\ipykernel_14596\3099160338.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  sinasc_nova.ESCMAE.fillna("Não Preenchido",inplace=True)


In [277]:
calcula_missing(sinasc_nova)

LOCNASC  -  0  missing(s). Equivale há 0.0% dos dados
IDADEMAE  -  0  missing(s). Equivale há 0.0% dos dados
ESTCIVMAE  -  0  missing(s). Equivale há 0.0% dos dados
ESCMAE  -  0  missing(s). Equivale há 0.0% dos dados
QTDFILVIVO  -  0  missing(s). Equivale há 0.0% dos dados
GESTACAO  -  0  missing(s). Equivale há 0.0% dos dados
GRAVIDEZ  -  0  missing(s). Equivale há 0.0% dos dados
CONSULTAS  -  0  missing(s). Equivale há 0.0% dos dados
APGAR5  -  0  missing(s). Equivale há 0.0% dos dados


8) O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização. 

In [278]:
# 8) seu código aqui
sinasc_nova.loc[sinasc_nova['APGAR5']>=8, "APGAR5_CAT"] = 'normal'
sinasc_nova.loc[(sinasc_nova['APGAR5']>=6) & (sinasc_nova['APGAR5']<=7), "APGAR5_CAT"] = 'asfixia leve'
sinasc_nova.loc[(sinasc_nova['APGAR5']>=4) & (sinasc_nova['APGAR5']<=5), "APGAR5_CAT"] = 'asfixia moderada'
sinasc_nova.loc[sinasc_nova['APGAR5']<=3, "APGAR5_CAT"] = 'asfixia severa'



In [279]:
sinasc_nova.head(25)

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5,APGAR5_CAT
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,normal
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0,normal
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0,normal
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0,normal
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0,normal
5,1,33,5.0,1 a 3 anos,2.0,37 a 41 semanas,Única,4,10.0,normal
6,1,37,1.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0,normal
7,1,27,2.0,12 anos ou mais,0.0,37 a 41 semanas,Única,4,10.0,normal
8,1,20,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,3,9.0,normal
9,1,30,2.0,1 a 3 anos,1.0,37 a 41 semanas,Única,4,10.0,normal


In [280]:
sinasc_nova['APGAR5_CAT'].value_counts()

APGAR5_CAT
normal              26463
asfixia leve          320
asfixia severa         74
asfixia moderada       68
Name: count, dtype: int64

In [281]:
# 9) seu código aqui
fn = lambda x:x.replace(" ","_").capitalize()
sinasc_nova['APGAR5_CAT'] = sinasc_nova['APGAR5_CAT'].apply(fn)
sinasc_nova.head(25)


,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5,APGAR5_CAT
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,Normal
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0,Normal
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0,Normal
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0,Normal
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0,Normal
5,1,33,5.0,1 a 3 anos,2.0,37 a 41 semanas,Única,4,10.0,Normal
6,1,37,1.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0,Normal
7,1,27,2.0,12 anos ou mais,0.0,37 a 41 semanas,Única,4,10.0,Normal
8,1,20,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,3,9.0,Normal
9,1,30,2.0,1 a 3 anos,1.0,37 a 41 semanas,Única,4,10.0,Normal
